In [9]:
import torch, os
from monai.networks.nets import SegResNet
from monai import transforms
from my_segmentation.ProstateSeg.dataset.prostateSeg import prostateSeg
from monai.data import DataLoader, decollate_batch


In [10]:
def _get_loader():
    file_root = "/Users/qixinhu/Project/CUHK/Prostate/PAIsData/0426/qixin/SEG"
    test_samples = []
    with open(os.path.join(file_root,'test_case_level.txt'), 'r') as f:
        lines = f.readlines()
        for line in lines:
            sample = os.path.join(file_root, line.strip() + '.npz')
            test_samples.append(sample)

    val_img_transform = transforms.Compose(
        [
            transforms.EnsureChannelFirst(channel_dim=0),
        ]
    )
    val_seg_transform = transforms.Compose(
        [
            transforms.EnsureChannelFirst(channel_dim=0),

        ]
    )

    test_ds = prostateSeg(npz_files=test_samples, modality=3, img_transforms=val_img_transform, seg_transforms=val_seg_transform)
    test_loader = DataLoader(test_ds, batch_size=1, shuffle=False,num_workers=0, pin_memory=True)

    return test_loader

In [11]:
test_loader = _get_loader()

In [12]:
test_batch = next(iter(test_loader))

In [19]:
test_batch['label'].shape

torch.Size([1, 1, 224, 224, 30])